## Hybrid properties & Computed columns

In SQLAlchemy, you can achieve the equivalent of Django's GeneratedField (which computes values using other columns) using hybrid properties or SQL expressions with computed columns. Both approaches are flexible and allow you to define computed fields that can operate at the Python or database level.

**Hybrid properties**

Produce SQL expressions at the class level and Python expression evaulation at the instance level

In [ ]:
from sqlalchemy import Computed, func
from sqlalchemy.orm import declarative_base, Mapped, mapped_column
from sqlalchemy.ext.hybrid import hybrid_property

Base = declarative_base()


class User(Base):
    __tablename__ = "users"

    id: Mapped[int] = mapped_column(primary_key=True)
    first_name: Mapped[str]
    last_name: Mapped[str]

    # Define a computed field
    @hybrid_property
    def full_name(self) -> str:  # type: ignore
        return f"{self.first_name} {self.last_name}"

    # Optional: Allow it to be used in queries too - SQL-level computed property for queries
    @full_name.expression
    def full_name(cls):
        return func.concat(cls.first_name, " ", cls.last_name)

**Computed Columns**

In [24]:
class User2(Base):
    __tablename__ = "users_2"

    id: Mapped[int] = mapped_column(primary_key=True)
    first_name: Mapped[str]
    last_name: Mapped[str]

    # Computed column at the database level
    full_name: Mapped[str] = mapped_column(Computed("first_name || ' ' || last_name"))

### Comparison Table: Hybrid Properties vs. Computed Columns vs. Column Properties

| Feature                      | **Hybrid Properties**                                                                                  | **Computed Columns**                                                                                  | **Column Properties**                                                                               |
|------------------------------|-------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------|
| **Definition**               | Defined at the Python level using `@hybrid_property`.                                                 | Defined at the database level using `Computed`.                                                      | Defined at the Python level using `column_property`.                                              |
| **Computation Location**     | Computed dynamically in Python or within SQL queries (with `.expression`).                           | Computed by the database engine during query execution.                                              | Computed as part of the SQL query, executed by the database.                                      |
| **Queryable**                | Yes, when combined with `.expression`, they can be used in SQLAlchemy queries.                       | Yes, directly queryable as part of the table schema.                                                 | Yes, directly queryable and treated like a standard column in SQLAlchemy queries.                |
| **Performance**              | Computation happens in Python (runtime) or SQL (if `.expression` is used).                          | Computation is handled entirely by the database, offloading processing from the application.         | Computation is handled entirely by the database at query time.                                   |
| **Flexibility**              | Supports complex Python logic (e.g., conditional computations, custom Python methods).              | Limited to SQL expressions supported by the database.                                                | Limited to SQL expressions but more efficient than hybrid properties for purely SQL-based logic. |
| **Persistence**              | Computed dynamically; values are not stored in the database.                                         | The column exists in the database schema but is read-only in SQLAlchemy.                             | Not persisted; values are computed dynamically during query execution.                           |
| **Database Dependency**      | Independent of the database. Computation logic resides in the application.                           | Database-dependent; SQL expressions must be compatible with the target database (e.g., SQLite vs. PostgreSQL). | Database-dependent; uses SQLAlchemy expressions that are translated to SQL.                      |
| **Read/Write Capability**    | Can be made writable by defining a custom setter in Python.                                          | Read-only; the database manages the value based on the defined SQL expression.                       | Read-only; derived values cannot be modified directly.                                            |
| **Type Checking**            | Fully integrated with Python typing, enhancing IDE support and type hints.                          | Requires the SQL expression to return a compatible type for the database column.                     | Inherits type from the SQLAlchemy expression or column used in the computation.                  |
| **Use Case**                 | Ideal for complex logic, application-layer computations, or scenarios where the database schema cannot be altered. | Ideal for simple computations, ensuring consistency at the database level (e.g., concatenation or arithmetic). | Ideal for lightweight, query-time computations that don’t require Python-side logic.             |


Object states

https://docs.sqlalchemy.org/en/20/orm/session_state_management.html